## Earth Rover Program - Penetration Resistance Scripts

1. Navigate to your conda env with conda activate {your env}
2. use pip install or conda install ant libraries that you do not have installed. 

Importing Libraries: Make sure to install the necessary libraries (pandas and openpyxl) if they are not already installed.

GitHub instructions
1. git status
2. git pull
3. git add -A
4. git commit -m "example commit message"
4. git push

Author: Joe Collins
Last update: 2024-12-06

In [19]:
# 01 Packages 
from pathlib import Path
import os
import pandas as pd
import numpy as np
import re
import geopandas as gpd
from pyproj import CRS

In [20]:
# print the current working directory to the screen
print('Get current working directory : ', os.getcwd())

Get current working directory :  /Users/u1764794/Library/CloudStorage/GoogleDrive-jcollins@earthroverprogram.org/Shared drives/DATA/FIELD_DEPLOYMENTS/2024-06-Joesfield


Change the WD to the deployment of interest

In [21]:
# set wd to googledrive 

os.chdir("/Users/u1764794/jcollins@earthroverprogram.org - Google Drive/Shared drives/DATA/FIELD_DEPLOYMENTS/2024-06-Joesfield/")

# print the current working directory to the screen
print('Get current working directory : ', os.getcwd())

print(os.listdir("/Users/u1764794/jcollins@earthroverprogram.org - Google Drive/Shared drives/DATA/FIELD_DEPLOYMENTS/2024-06-Joesfield/"))


Get current working directory :  /Users/u1764794/Library/CloudStorage/GoogleDrive-jcollins@earthroverprogram.org/Shared drives/DATA/FIELD_DEPLOYMENTS/2024-06-Joesfield
['session_0012', 'session_0015', "S13_L3c_Joe's field_no till.png", 'session_0014', 'session_0013', "S3_L3c_Joe's field_no till.png", "S5_L4b_Joe's field_till.png", "S4_L4c_Joe's field_till.png", '.DS_Store', "S12_L2c_Joe's field_no till.png", "S11_L1a_Joe's field_till.png", "S6_L4a_Joe's field_till.png", 'session_0007', 'session_0009', 'session_0008', 'session_0004.zip', "S1_L3a_Joe's field_no till.png", 'session_0006', 'session_0001', 'session_0016', 'session_0011', 'session_0010', "S9_L1c_Joe's field_till.png", "S14_L4a_Joe's field_till.png", "S10_L1b_Joe's field_till.png", "S8_L2b_Joe's field_no till.png", "S16_LNone_Joe's field_no till.png", 'index.json', 'soil_data', 'session_0004', 'session_0003', "S15_L4a_Joe's field_till.png", "S2_L3b_Joe's field_no till.png", "S7_L2a_Joe's field_no till.png", 'session_0002', 's

Set the input directory.

THIS SHOULD BE WHERE THE PENETROLOGGER TEXT FILES ARE CONTAINED FROM THE DEPLOYMENT OF INTEREST. 

In [22]:
# set the path for the txt files
input_dir = "soil_data/penetration_res/raw_data/txt/"

# Ensure directories exist
Path(input_dir).mkdir(parents=True, exist_ok=True)

# print the filelist in the input_dir
print(os.listdir(input_dir))

['.DS_Store', '24060703.txt', '24060706.txt', '24060707.txt', '24060705.txt', '24060704.txt', '24053102.txt']


Output dir 

Create a folder for the processed files and set it as the output dir

In [23]:
# Specify the directory path
dir_path = "soil_data/penetration_res/processed_data/short_format/"

# Create the directory
os.makedirs(dir_path, exist_ok=True)  # exist_ok=True prevents an error if the directory already exists

# set the path for the txt files
output_short_dir = "soil_data/penetration_res/processed_data/short_format/"

# Ensure directories exist
Path(output_short_dir).mkdir(parents=True, exist_ok=True)

# print the filelist in the input_dir
print(os.listdir(output_short_dir))

['24060706_processed.csv', '24060705_processed.csv', '24060704_processed.csv', '24060703_processed.csv', '24053102_processed.csv', '24060707_processed.csv']


Long-format processing 

This long format data is better for plotting the data with (at least on R anayway!! MAYBE THIS STEP IS NOT NEEDED ON PYTHON???? TINA PLEASE HELP ME? HAHA!)

In [24]:
# Specify the directory path
dir_path = "soil_data/penetration_res/processed_data/long_format/"

# Create the directory
os.makedirs(dir_path, exist_ok=True)  # exist_ok=True prevents an error if the directory already exists

# set the path for the txt files
output_long_format_dir = "soil_data/penetration_res/processed_data/long_format/"

# Ensure directories exist
Path(output_long_format_dir).mkdir(parents=True, exist_ok=True)

# print the filelist in the input_dir
print(os.listdir(output_long_format_dir))

['24060704_processed_data.csv', '24053102_processed_data.csv', '24060707_processed_data.csv', '24060706_processed_data.csv', '24060705_processed_data.csv', '24060703_processed_data.csv']


DATA INFORMATION

Create data info files and save them to a new folder

In [25]:
# Specify the directory path
dir_path = "soil_data/penetration_res/data_info/"

# Create the directory
os.makedirs(dir_path, exist_ok=True)  # exist_ok=True prevents an error if the directory already exists

# set the path for the txt files
info_dir = "soil_data/penetration_res/data_info/"

# Ensure directories exist
Path(info_dir).mkdir(parents=True, exist_ok=True)

# print the filelist in the input_dir
print(os.listdir(info_dir))

['24060705_processed_info.csv', '.DS_Store', '24060706_processed_info.csv', 'meta_info.csv', '24060703_processed_info.csv', '24060707_processed_info.csv', '24060704_processed_info.csv', '24053102_processed_info.csv']


LOAD THE RAW DATA

create a list of txt files in the raw data folder and read them all in as a list

In [54]:
# 1. Load Data
## Get the file list
filelist = list(Path(input_dir).glob("*.txt"))
## Create empty lists to store the data frames from each file
file_data = []
file_info = []

# Loop through each file in the file list
for file in filelist:
    try:
        # Read penetration resistance data
        data = pd.read_csv(
            file,
            sep="\t",
            header=None,
            skiprows=14,
            names=["name", "coordinates"] + [f"X{i}" for i in range(1, 83)],
            decimal=","
        )
        file_data.append(data)
        
        # Read metadata (info) from the same file
        info = pd.read_csv(
            file,
            sep=":",
            header=None,
            skiprows=2,
            nrows=9,
            names=["name", "data"]
        )
        file_info.append(info)
    except Exception as e:
        print(f"Error processing {file}: {e}")


        # Check the first few rows of the data
print(file_data[0].head())  # Check penetration resistance data for the first file
print(file_info[0])        # Check metadata for the first file


         name             coordinates  X1    X2    X3    X4    X5    X6    X7  \
0  PLOT-001.1  N52 54.960 W002 36.372   0   0.4   0.7   0.9   0.9   0.8   0.8   
1  PLOT-001.2  N52 54.961 W002 36.371   0   0.2   0.2   1.0   0.8   0.8   0.8   
2  PLOT-001.3  N52 54.961 W002 36.370   0   0.2   0.2   0.2   0.9   1.1   1.1   
3  PLOT-001.4  N52 54.962 W002 36.368   0   0.2   0.9   0.9   0.9   0.8   0.8   
4  PLOT-001.5  N52 54.962 W002 36.368   0   1.1   1.2   1.3   1.4   1.4   1.4   

     X8  ...   X73   X74   X75   X76   X77   X78   X79   X80   X81   X82  
0   0.8  ...   2.4   2.4   2.4   2.6   2.6   2.4   2.4   2.4   2.0   2.0  
1   0.7  ...   2.0   2.0   2.0   2.0   2.0   1.9   2.0   2.0   2.0   2.8  
2   1.1  ...   1.9   1.8   1.8   1.8   1.8   1.8   1.8   1.9   1.9   2.1  
3   0.8  ...   2.5   3.1   3.1   3.1   3.1   3.1   3.1   3.1   3.1   3.1  
4   1.4  ...   1.7   1.7   1.7   1.7   2.1   2.5   2.5   2.6   2.6   2.6  

[5 rows x 84 columns]
                  name             data


In [55]:
print(file_data[0].head())  # Check penetration resistance data for the first file


         name             coordinates  X1    X2    X3    X4    X5    X6    X7  \
0  PLOT-001.1  N52 54.960 W002 36.372   0   0.4   0.7   0.9   0.9   0.8   0.8   
1  PLOT-001.2  N52 54.961 W002 36.371   0   0.2   0.2   1.0   0.8   0.8   0.8   
2  PLOT-001.3  N52 54.961 W002 36.370   0   0.2   0.2   0.2   0.9   1.1   1.1   
3  PLOT-001.4  N52 54.962 W002 36.368   0   0.2   0.9   0.9   0.9   0.8   0.8   
4  PLOT-001.5  N52 54.962 W002 36.368   0   1.1   1.2   1.3   1.4   1.4   1.4   

     X8  ...   X73   X74   X75   X76   X77   X78   X79   X80   X81   X82  
0   0.8  ...   2.4   2.4   2.4   2.6   2.6   2.4   2.4   2.4   2.0   2.0  
1   0.7  ...   2.0   2.0   2.0   2.0   2.0   1.9   2.0   2.0   2.0   2.8  
2   1.1  ...   1.9   1.8   1.8   1.8   1.8   1.8   1.8   1.9   1.9   2.1  
3   0.8  ...   2.5   3.1   3.1   3.1   3.1   3.1   3.1   3.1   3.1   3.1  
4   1.4  ...   1.7   1.7   1.7   1.7   2.1   2.5   2.5   2.6   2.6   2.6  

[5 rows x 84 columns]


In [56]:
print(file_data[0])  # Check penetration resistance data for the first file


           name             coordinates  X1    X2    X3    X4    X5    X6  \
0    PLOT-001.1  N52 54.960 W002 36.372   0   0.4   0.7   0.9   0.9   0.8   
1    PLOT-001.2  N52 54.961 W002 36.371   0   0.2   0.2   1.0   0.8   0.8   
2    PLOT-001.3  N52 54.961 W002 36.370   0   0.2   0.2   0.2   0.9   1.1   
3    PLOT-001.4  N52 54.962 W002 36.368   0   0.2   0.9   0.9   0.9   0.8   
4    PLOT-001.5  N52 54.962 W002 36.368   0   1.1   1.2   1.3   1.4   1.4   
5    PLOT-001.6  N52 54.962 W002 36.367   0   0.7   1.4   2.3   2.2   2.0   
6    PLOT-001.7  N52 54.964 W002 36.365   0   0.2   0.7   0.7   0.7   0.7   
7    PLOT-001.8  N52 54.963 W002 36.362   0   2.0   2.0   1.9   1.3   1.3   
8    PLOT-001.9  N52 54.963 W002 36.361   0   0.8   1.2   1.2   1.2   1.1   
9   PLOT-001.10  N52 54.964 W002 36.359   0   1.2   2.1   2.1   1.5   1.5   
10  PLOT-001.11  N52 54.964 W002 36.358   0   0.6   2.5   2.5   2.5   2.2   
11  PLOT-001.12  N52 54.964 W002 36.356   0   1.2   2.1   2.1   2.5   2.8   

PROCESS THE RAW DATA

This removes the information from the top of the txt file and save them to csv files. 

In [57]:
# 2. Save Processed Data
# Loop through the list of dataframes
for i, data in enumerate(file_data):
    basename = Path(filelist[i]).stem
    output_path = f"{output_long_format_dir}{basename}_processed_data.csv"  # Name the CSV file with path
    data.to_csv(output_path, index=False)
    print(f"Saved processed data for {basename} to {output_path}")

Saved processed data for 24060703 to soil_data/penetration_res/processed_data/long_format/24060703_processed_data.csv
Saved processed data for 24060706 to soil_data/penetration_res/processed_data/long_format/24060706_processed_data.csv
Saved processed data for 24060707 to soil_data/penetration_res/processed_data/long_format/24060707_processed_data.csv
Saved processed data for 24060705 to soil_data/penetration_res/processed_data/long_format/24060705_processed_data.csv
Saved processed data for 24060704 to soil_data/penetration_res/processed_data/long_format/24060704_processed_data.csv
Saved processed data for 24053102 to soil_data/penetration_res/processed_data/long_format/24053102_processed_data.csv


Create separate info files for each .pen project and save to csv

In [58]:
# 3. Save Processed info
# Loop through the list of metadata dataframes
for i, info in enumerate(file_info):
    basename = Path(filelist[i]).stem
    output_path = f"{info_dir}{basename}_processed_info.csv"  # Name the CSV file with path
    info.to_csv(output_path, index=False)
    print(f"Saved metadata for {basename} to {output_path}")

Saved metadata for 24060703 to soil_data/penetration_res/data_info/24060703_processed_info.csv
Saved metadata for 24060706 to soil_data/penetration_res/data_info/24060706_processed_info.csv
Saved metadata for 24060707 to soil_data/penetration_res/data_info/24060707_processed_info.csv
Saved metadata for 24060705 to soil_data/penetration_res/data_info/24060705_processed_info.csv
Saved metadata for 24060704 to soil_data/penetration_res/data_info/24060704_processed_info.csv
Saved metadata for 24053102 to soil_data/penetration_res/data_info/24053102_processed_info.csv


ADD TREATMENTS TO THE INFO FILES

*** YOU NEED TO EDIT THIS HERE DEPENDING ON THE TREATMENTS WITHIN THE SPECIFIC DEPLOYMENT ***

In [59]:
# 4. Add Treatments to info
# Add treatments based on the project number in the metadata
for i, info in enumerate(file_info):
    # Check if the first value in the 'data' column matches 24060703 or 24060704
    treatment = "Conventional" if int(info.iloc[0, 1]) in [24060703, 24060704] else "Conservation"
    
    # Add the treatment as a new row
    new_row = pd.DataFrame([{"name": "Treatment", "data": treatment}])
    file_info[i] = pd.concat([info, new_row], ignore_index=True)

    print(file_info[i])

                  name             data
0  Serial number             \t42323710
1  Projectname               \t24060703
2  Username                  \tERP     
3  Plotdate                  \t07-06-24
4  Nr. of pen/plot                 \t14
5  Nr of pen done                  \t14
6  Cone type            \t2.0cm2\t60deg
7  Penetrationspeed           \t2\tcm/s
8  Depth meas unit                   cm
9            Treatment     Conservation
                  name             data
0  Serial number             \t42323710
1  Projectname               \t24060706
2  Username                  \tERP     
3  Plotdate                  \t07-06-24
4  Nr. of pen/plot                 \t12
5  Nr of pen done                  \t12
6  Cone type            \t2.0cm2\t60deg
7  Penetrationspeed           \t2\tcm/s
8  Depth meas unit                   cm
9            Treatment     Conservation
                  name             data
0  Serial number             \t42323710
1  Projectname               \t24060707


In [60]:
# 5. Verify File Info
file_info[0] = pd.concat([file_info[0], pd.DataFrame([{"name": "Treatment", "data": "Conservation"}])], ignore_index=True)

print(file_info[0])

                   name             data
0   Serial number             \t42323710
1   Projectname               \t24060703
2   Username                  \tERP     
3   Plotdate                  \t07-06-24
4   Nr. of pen/plot                 \t14
5   Nr of pen done                  \t14
6   Cone type            \t2.0cm2\t60deg
7   Penetrationspeed           \t2\tcm/s
8   Depth meas unit                   cm
9             Treatment     Conservation
10            Treatment     Conservation


Add the coordinates and compile a metadata file

In [33]:
# 6. Coordinates and Metadata Processing
# Initialize an empty DataFrame
coord_dat = pd.DataFrame()

# Loop through each data frame in file_data
for i in range(len(file_data)):
    # Dynamically add metadata columns from file_info
    file_data[i]['project'] = file_info[i].iloc[0, 1]
    file_data[i]['username'] = file_info[i].iloc[1, 1]
    file_data[i]['date'] = file_info[i].iloc[2, 1]
    file_data[i]['pens_per_plot'] = file_info[i].iloc[3, 1]
    file_data[i]['nr_of_pens_done'] = file_info[i].iloc[4, 1]
    file_data[i]['cone_type'] = file_info[i].iloc[5, 1]
    file_data[i]['pen_speed'] = file_info[i].iloc[6, 1]
    file_data[i]['depth_unit'] = file_info[i].iloc[7, 1]
    file_data[i]['pressure_unit'] = file_info[i].iloc[8, 1]

    # Ensure columns are ordered as per R script logic
    reordered_columns = ['name', 'coordinates', 'project', 'username', 'date', 
                         'pens_per_plot', 'nr_of_pens_done', 'cone_type', 
                         'pen_speed', 'depth_unit', 'pressure_unit']
    
    # Dynamically filter and reorder columns
    file_data[i] = file_data[i][[col for col in reordered_columns if col in file_data[i].columns]]

    # Append rows to coord_dat
    coord_dat = pd.concat([coord_dat, file_data[i]], ignore_index=True)

# Function to convert DMS to decimal degrees
def dms_to_decimal(deg, min_, direction):
    try:
        decimal = deg + min_ / 60
        if direction in ['S', 'W']:
            decimal = -decimal
        return decimal
    except TypeError:
        return None

# Extract and convert coordinates
coord_dat['lat_deg'] = coord_dat['coordinates'].str.extract(r'(?<=N|S)(\d+)').astype(float)
coord_dat['lat_min'] = coord_dat['coordinates'].str.extract(r'(?<=N\d{2} )(\d+\.\d+)').astype(float)
coord_dat['lon_deg'] = coord_dat['coordinates'].str.extract(r'(?<=W|E)(\d+)').astype(float)
coord_dat['lon_min'] = coord_dat['coordinates'].str.extract(r'(?<=W\d{3} )(\d+\.\d+)').astype(float)
coord_dat['lat_dir'] = coord_dat['coordinates'].str.extract(r'^([NS])')
coord_dat['lon_dir'] = coord_dat['coordinates'].str.extract(r'([WE])$')
coord_dat['latitude'] = coord_dat.apply(lambda x: dms_to_decimal(x['lat_deg'], x['lat_min'], x['lat_dir']), axis=1)
coord_dat['longitude'] = coord_dat.apply(lambda x: dms_to_decimal(x['lon_deg'], x['lon_min'], x['lon_dir']), axis=1)

# Extract plot number
coord_dat['plot_number'] = coord_dat['name'].str.extract(r'(?<=PLOT-)(\d+\.\d+)').astype(float)

# Change CRS
coord_dat['longitude'] = coord_dat['longitude'] * -1

"""# Add the treatment column
coord_dat['treatment'] = coord_dat['project'].apply(
    lambda x: "Conservation" if x == 24060703 or x == 24060704 else "Conventional"
)"""

# Remove rows with "PENETRATION DATA"
coord_dat = coord_dat[~coord_dat['name'].str.contains('PENETRATION DATA', na=False)]

# Fill missing values with 'NA'
coord_dat = coord_dat.fillna('NA')


# Save as CSV
coord_dat.to_csv(f"{info_dir}meta_info.csv", index=False)

In [34]:
print(file_data)

df = pd.DataFrame(file_data)

# Print the first 5 rows (default)
print(df.head())

[                name             coordinates     project    username  \
0   PENETRATION DATA                     NaN  \t42323710  \t24060703   
1         PLOT-001.1  N52 54.960 W002 36.372  \t42323710  \t24060703   
2         PLOT-001.2  N52 54.961 W002 36.371  \t42323710  \t24060703   
3         PLOT-001.3  N52 54.961 W002 36.370  \t42323710  \t24060703   
4         PLOT-001.4  N52 54.962 W002 36.368  \t42323710  \t24060703   
5         PLOT-001.5  N52 54.962 W002 36.368  \t42323710  \t24060703   
6         PLOT-001.6  N52 54.962 W002 36.367  \t42323710  \t24060703   
7         PLOT-001.7  N52 54.964 W002 36.365  \t42323710  \t24060703   
8         PLOT-001.8  N52 54.963 W002 36.362  \t42323710  \t24060703   
9         PLOT-001.9  N52 54.963 W002 36.361  \t42323710  \t24060703   
10       PLOT-001.10  N52 54.964 W002 36.359  \t42323710  \t24060703   
11       PLOT-001.11  N52 54.964 W002 36.358  \t42323710  \t24060703   
12       PLOT-001.12  N52 54.964 W002 36.356  \t42323710  \t240

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (6,) + inhomogeneous part.

In [35]:
for file in filelist:
    try:
        # Read penetration resistance data
        data = pd.read_csv(
            file,
            sep="\t",
            header=None,
            skiprows=13,
            names=["name", "coordinates"] + [f"X{i}" for i in range(1, 83)],
            decimal=","
        )
        file_data.append(data)



SyntaxError: incomplete input (2023032648.py, line 13)

In [36]:
## Get the file list
filelist = list(Path(input_dir).glob("*.txt"))

# Create an empty dictionary to store the data frames from each file
file_data = {}

# Loop through each file in the file list
for file in filelist:
    # Read the file using pandas read_csv
    data = pd.read_csv(
        file,
        sep="\t",  # Tab delimiter
        header=0,  # First row is the header
        decimal=",",  # Comma as the decimal separator
        names=["name", "coordinates"] + list(range(1, 83)),  # Column names
        skiprows=13  # Skip the first 13 rows
    )
    
    # Store the data in the dictionary
    file_data[file] = data

    # Print the file name and its corresponding data
    print(f"Data from file: {file}")
    print(data)
    print("\n" + "="*80 + "\n")  # Separator for better readability


Data from file: soil_data/penetration_res/raw_data/txt/24060703.txt
           name             coordinates  1     2     3     4     5     6  \
0    PLOT-001.1  N52 54.960 W002 36.372  0   0.4   0.7   0.9   0.9   0.8   
1    PLOT-001.2  N52 54.961 W002 36.371  0   0.2   0.2   1.0   0.8   0.8   
2    PLOT-001.3  N52 54.961 W002 36.370  0   0.2   0.2   0.2   0.9   1.1   
3    PLOT-001.4  N52 54.962 W002 36.368  0   0.2   0.9   0.9   0.9   0.8   
4    PLOT-001.5  N52 54.962 W002 36.368  0   1.1   1.2   1.3   1.4   1.4   
5    PLOT-001.6  N52 54.962 W002 36.367  0   0.7   1.4   2.3   2.2   2.0   
6    PLOT-001.7  N52 54.964 W002 36.365  0   0.2   0.7   0.7   0.7   0.7   
7    PLOT-001.8  N52 54.963 W002 36.362  0   2.0   2.0   1.9   1.3   1.3   
8    PLOT-001.9  N52 54.963 W002 36.361  0   0.8   1.2   1.2   1.2   1.1   
9   PLOT-001.10  N52 54.964 W002 36.359  0   1.2   2.1   2.1   1.5   1.5   
10  PLOT-001.11  N52 54.964 W002 36.358  0   0.6   2.5   2.5   2.5   2.2   
11  PLOT-001.12  N52

In [45]:
# Assuming file_data is a dictionary where keys are filenames and values are DataFrames
# Assuming file_info is a list of metadata dictionaries corresponding to each file
# Replace these variables with your actual data and metadata

# Process each DataFrame in file_data
for i, (filename, df) in enumerate(file_data.items()):
    # Remove the 1st to 11th columns
    df = df.iloc[:, 2:]

    # Convert all columns to numeric (coercing errors to NaN)
    df = df.apply(pd.to_numeric, errors='coerce')

    # Transpose the DataFrame
    df = df.transpose()

    # Add the mean row
    df['mean'] = df.mean(axis=1)

    # Calculate standard deviation for each row
    df['stdev'] = df.iloc[:, :-1].std(axis=1)

    # Calculate standard error of the mean (SEM) for each row
    df['sem'] = df.iloc[:, :-1].apply(lambda x: x.std(ddof=1) / np.sqrt(x.notna().sum()), axis=1)

    # Create depth DataFrame
    depth = pd.DataFrame({'depth_cm': range(1, 83)})

    # Bind depth DataFrame with the processed DataFrame
    df = pd.concat([depth, df.reset_index(drop=True)], axis=1)

    # Add project name from file_info
    df['project_name'] = file_info[i]['data'][1]

    # Add company name from file_info
    df['company_name'] = file_info[i]['data'][2]

    # Add date from file_info
    df['date'] = file_info[i]['data'][3]

    # Add cone type from file_info
    df['cone_type'] = file_info[i]['data'][6]

    # Add pen speed from file_info
    df['pen_speed'] = file_info[i]['data'][7]

    # Generate the output CSV filename
    csv_file = os.path.join(output_short_dir, f"{os.path.splitext(os.path.basename(filename))[0]}_processed.csv")

    # Save the processed DataFrame as a CSV
    df.to_csv(csv_file, index=False)

    # Print a message indicating the file was saved
    print(f"Saved {csv_file}")

Saved soil_data/penetration_res/processed_data/short_format/24060703_processed.csv
Saved soil_data/penetration_res/processed_data/short_format/24060706_processed.csv
Saved soil_data/penetration_res/processed_data/short_format/24060707_processed.csv
Saved soil_data/penetration_res/processed_data/short_format/24060705_processed.csv
Saved soil_data/penetration_res/processed_data/short_format/24060704_processed.csv
Saved soil_data/penetration_res/processed_data/short_format/24053102_processed.csv


In [43]:
file_info[i]['data'][4]

'\t5'

*** LONG FORMAT PROCESSING AND BIND ***

This step is to plot the data in 2D along an instrument array. 

In [64]:
import os
import pandas as pd
import re
import numpy as np

# Function to convert DMS to decimal format
def dms_to_decimal(degrees, minutes, direction):
    decimal = degrees + (minutes / 60.0)
    if direction in ['S', 'W']:
        decimal = -decimal
    return decimal

# Transpose function to convert data into long format
def transpose_data(df):
    # Melt the DataFrame to long format
    df_melted = pd.melt(
        df, 
        id_vars=["name", "coordinates"], 
        var_name="depth", 
        value_name="penetration_resistance"
    )
    
    # Remove the prefix "X" from the depth column and convert to numeric
    df_melted['depth'] = df_melted['depth'].str.replace('X', '').astype(float)
    
    # Order the data
    df_melted = df_melted.sort_values(by=["name", "coordinates", "depth"])
    
    # Select relevant columns
    df_transposed = df_melted[["name", "coordinates", "depth", "penetration_resistance"]]
    return df_transposed



# File path and list
file_path = "soil_data/penetration_res/processed_data/long_format/"
filelist = [os.path.join(file_path, f) for f in os.listdir(file_path) if f.endswith(".csv")]

# Read files and process them
file_data = []
for file in filelist:
    data = pd.read_csv(file)
    file_data.append(transpose_data(data))

# Combine all dataframes into one
long_data = pd.concat(file_data, ignore_index=True)

# Extract and convert coordinates
long_data['lat_deg'] = long_data['coordinates'].str.extract(r'(?<=N|S)(\d+)').astype(float)
long_data['lat_min'] = long_data['coordinates'].str.extract(r'(?<=N\d{2} )(\d+\.\d+)').astype(float)
long_data['lon_deg'] = long_data['coordinates'].str.extract(r'(?<=W|E)(\d+)').astype(float)
long_data['lon_min'] = long_data['coordinates'].str.extract(r'(?<=W\d{3} )(\d+\.\d+)').astype(float)
# Corrected regex to include capture groups
long_data['lat_dir'] = long_data['coordinates'].str.extract(r'^([NS])')  # Capture N or S
long_data['lon_dir'] = long_data['coordinates'].str.extract(r'([WE])$')  # Capture W or E


long_data['latitude'] = long_data.apply(
    lambda row: dms_to_decimal(row['lat_deg'], row['lat_min'], row['lat_dir']), axis=1
)
long_data['longitude'] = long_data.apply(
    lambda row: dms_to_decimal(row['lon_deg'], row['lon_min'], row['lon_dir']), axis=1
)

# Extract plot number and replicate number from 'name' column
long_data['plot_number'] = long_data['name'].str.extract(r'PLOT-(\d{3})')
long_data['replicate_number'] = long_data['name'].str.extract(r'PLOT-\d{3}\.(\d)')

# Save the final data to a CSV file
output_file = "soil_data/penetration_res/processed_data/long_format_data.csv"
long_data.to_csv(output_file, index=False)

print(f"Data saved to {output_file}")

long_data


Data saved to soil_data/penetration_res/processed_data/long_format_data.csv


,name,coordinates,depth,penetration_resistance,lat_deg,lat_min,lon_deg,lon_min,lat_dir,lon_dir,latitude,longitude,plot_number,replicate_number
0,PLOT-001.1,N52 54.960 W002 36.368,1.0,0.0,52.0,54.960,2.0,36.368,N,NaN,52.91600,2.606133,001,1
1,PLOT-001.1,N52 54.960 W002 36.368,2.0,0.2,52.0,54.960,2.0,36.368,N,NaN,52.91600,2.606133,001,1
2,PLOT-001.1,N52 54.960 W002 36.368,3.0,0.2,52.0,54.960,2.0,36.368,N,NaN,52.91600,2.606133,001,1
3,PLOT-001.1,N52 54.960 W002 36.368,4.0,0.2,52.0,54.960,2.0,36.368,N,NaN,52.91600,2.606133,001,1
4,PLOT-001.1,N52 54.960 W002 36.368,5.0,0.2,52.0,54.960,2.0,36.368,N,NaN,52.91600,2.606133,001,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6227,PLOT-001.9,N52 54.963 W002 36.361,78.0,2.0,52.0,54.963,2.0,36.361,N,NaN,52.91605,2.606017,001,9
6228,PLOT-001.9,N52 54.963 W002 36.361,79.0,2.0,52.0,54.963,2.0,36.361,N,NaN,52.91605,2.606017,001,9
6229,PLOT-001.9,N52 54.963 W002 36.361,80.0,2.0,52.0,54.963,2.0,36.361,N,NaN,52.91605,2.606017,001,9
6230,PLOT-001.9,N52 54.963 W002 36.361,81.0,2.0,52.0,54.963,2.0,36.361,N,NaN,52.91605,2.606017,001,9


*** INSTRUMENT LOCATIONS ***

load and change crs of instrument locations

In [ ]:
# LOAD THE INSTRUMENT LOCATION DATA

# Define the input and output file paths
input_file = "soil_data/penetration_res/raw_data/stryde_deployed_positions.csv"
output_file = "soil_data/penetration_res/processed_data/stryde_positions_crs_4326.csv"

In [ ]:

# Read the CSV file
stryde_positions = pd.read_csv(input_file)

# Ensure Easting and Northing columns are numeric
stryde_positions['Easting'] = pd.to_numeric(stryde_positions['Easting'], errors='coerce')
stryde_positions['Northing'] = pd.to_numeric(stryde_positions['Northing'], errors='coerce')

# Drop rows with missing coordinates
stryde_positions = stryde_positions.dropna(subset=['Easting', 'Northing'])

# Initialize the transformer for UTM to WGS84 (latitude/longitude)
transformer = Transformer.from_crs("EPSG:32630", "EPSG:4326", always_xy=True)

# Transform Easting/Northing to longitude/latitude
coords = transformer.transform(stryde_positions['Easting'].values, stryde_positions['Northing'].values)
stryde_positions['longitude'], stryde_positions['latitude'] = coords

# Save the transformed coordinates to a new CSV file
stryde_positions.to_csv(output_file, index=False)

print(f"Transformed coordinates saved to: {output_file}")


Transformed coordinates saved to: soil_data/penetration_res/processed_data/stryde_positions_crs_4326.csv
